In [1]:
import pandas as pd

## 데이터 불러오기

In [2]:
df = pd.read_csv('https://github.com/euphoris/datasets/raw/master/imdb.zip')

In [3]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
df.shape

(1000, 2)

## 단어 문서 행렬 만들기

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features= 1000, stop_words='english')
tdm = cv.fit_transform(df['review'])

## 데이터분할 (train, test split)

In [6]:
from sklearn.model_selection import train_test_split

X = tdm #단어 컬럼
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.2, random_state = 1111)


In [8]:
print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

(800, 1000) (800,)
(200, 1000) (200,)


## 로지스틱 회귀분석 with 텐서플로우

In [10]:
# 설치
!pip install tensorflow

     |████████████████████████████████| 165.2 MB 7.0 MB/s eta 0:00:01   |█                               | 5.6 MB 4.1 MB/s eta 0:00:40     |███                             | 15.2 MB 4.9 MB/s eta 0:00:31     |█████▎                          | 27.0 MB 3.3 MB/s eta 0:00:42     |█████████                       | 46.0 MB 4.9 MB/s eta 0:00:25     |██████████▍                     | 53.4 MB 10.5 MB/s eta 0:00:11     |████████████▉                   | 66.4 MB 2.4 MB/s eta 0:00:42     |██████████████▍                 | 74.5 MB 1.8 MB/s eta 0:00:51     |███████████████▏                | 78.0 MB 1.8 MB/s eta 0:00:49     |████████████████████▏           | 103.9 MB 3.5 MB/s eta 0:00:18     |████████████████████████▉       | 128.3 MB 3.1 MB/s eta 0:00:12
     |████████████████████████████████| 65 kB 5.5 MB/s eta 0:00:011
     |████████████████████████████████| 42 kB 2.1 MB/s eta 0:00:011
     |████████████████████████████████| 10.6 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 M

In [20]:
import tensorflow as tf

### 모델 생성

In [21]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(
    1,  # 출력은 긍정(1)/부정(0) 하나
    input_shape=(1000,), # 입력은 단어 수
    activation='sigmoid'))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 1001      
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### 학습

In [31]:
model.fit(X_train.toarray(), y_train.values, batch_size=32, epochs=50)

Epoch 1/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5961 - accuracy: 0.8700
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5882 - accuracy: 0.8775
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5805 - accuracy: 0.8825
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5731 - accuracy: 0.8888
Epoch 5/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5659 - accuracy: 0.8875
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5588 - accuracy: 0.8950
Epoch 7/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5521 - accuracy: 0.8988
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5454 - accuracy: 0.9000
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5389 - accuracy: 0.9013
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5326 - accuracy: 0.9038
Epoch 11/50
25/25 [

### 테스트

In [32]:
model.evaluate(X_test.toarray(),y_test)

7/7 [==============================] - 0s 924us/step - loss: 0.4994 - accuracy: 0.7950


[0.4993995428085327, 0.7950000166893005]

## 모델 저장

In [33]:
model.save('imdb-sentiment.krs')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: imdb-sentiment.krs/assets


## 단어별 가중치 확인

In [35]:
weight, bias = model.trainable_weights

In [36]:
word_weight = pd.DataFrame({
    '단어':cv.get_feature_names(),
    '가중치': weight.numpy().flat
    
})

In [39]:
#부정적인 단어
word_weight.sort_values('가중치').head(20)

,단어,가중치
69,bad,-0.884455
68,awful,-0.774360
986,worst,-0.703701
842,stupid,-0.696910
763,script,-0.615045
134,cheap,-0.593982
658,plot,-0.591006
950,wasted,-0.585696
827,started,-0.582100
413,holes,-0.578451


In [41]:
#긍정적인 단어
word_weight.sort_values('가중치',ascending=False).head(20)

,단어,가중치
529,love,0.713425
503,liked,0.694925
382,great,0.694204
594,nice,0.650863
530,loved,0.649958
976,wonderful,0.636237
292,excellent,0.622293
76,beautiful,0.594056
409,hilarious,0.580077
449,interesting,0.569849
